In [ ]:
import os
import urllib.request
import librosa
import soundfile
import numpy as np
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename
import pickle
import pandas as pd
import pyaudio
import wave
import time

with open(f'model/final.pkl', 'rb') as f:
    model = pickle.load(f)

UPLOAD_FOLDER = './uploads'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['wav'])

def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('upload.html', intro=True)

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('Invalid File')
            return render_template('upload.html', intro=False)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            flash('File successfully uploaded')
            feat = extract_feature(myfile)
            feat = feat.reshape(1,-1)
            prediction = model.predict(feat)
            prediction = prediction[0].split('_')
            image_file = os.path.join('./static', f'{prediction[1]}-{prediction[0]}.jpg')
            return  render_template('upload.html', result=True, gender=prediction[1], mood=prediction[0], image_file=image_file)
        else:
            flash('Allowed file type is just wav for now')
            return render_template('upload.html', intro=False)

@app.route('/a', methods=['POST'])
def record():
    moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 2
    fs = 44100  # Record at 44100 samples per second
    seconds = 5
    filename = f'{moment}.wav'

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk, input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(os.path.join(app.config['UPLOAD_FOLDER'], filename), 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    feat = extract_feature(myfile)
    feat = feat.reshape(1,-1)
    prediction = model.predict(feat)
    prediction = prediction[0].split('_')
    image_file = os.path.join('./static', f'{prediction[1]}-{prediction[0]}.jpg')
    return  render_template('upload.html', result=True, gender=prediction[1], mood=prediction[0], image_file=image_file)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2020 21:12:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:12:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:12:49] "GET /favicon.ico HTTP/1.1" 404 -
/Applications/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
127.0.0.1 - - [11/Apr/2020 21:13:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:13:28] "GET / HTTP/1.1" 200 -


Recording


127.0.0.1 - - [11/Apr/2020 21:13:39] "POST /a HTTP/1.1" 200 -


Finished recording


...

127.0.0.1 - - [11/Apr/2020 21:14:17] "POST /a HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:14:17] "GET /a HTTP/1.1" 405 -


Recording
Finished recording


127.0.0.1 - - [11/Apr/2020 21:14:24] "POST /a HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:14:24] "GET /a HTTP/1.1" 405 -


Recording
Finished recording


127.0.0.1 - - [11/Apr/2020 21:14:34] "POST /a HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:14:34] "GET /a HTTP/1.1" 405 -
127.0.0.1 - - [11/Apr/2020 21:15:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:15:05] "GET / HTTP/1.1" 200 -
/Applications/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
127.0.0.1 - - [11/Apr/2020 21:23:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:23:51] "GET /static/male-afraid.jpg HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:23:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:09] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:14] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 21:24:19] "GET /stat